In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import cv2
import yaml
import open3d as o3d
from tqdm.notebook import tqdm
import random

random.seed(42)
np.random.seed(42)
cv2.setRNGSeed(42)

# Add the project directory to the Python path
sys.path.append('..')

# Import project modules
from src.preprocessing.image_loader import load_image_sequence, resize_images
from src.preprocessing.image_preprocessing import enhance_contrast
from src.feature_extraction.sift_extractor import extract_features_from_image_set
from src.feature_extraction.orb_extractor import extract_distributed_orb_features, extract_orb_features
from src.feature_extraction.feature_matcher import match_image_pairs, geometric_verification
from src.sfm.camera_calibration import estimate_camera_matrix
from src.sfm.pose_estimation import estimate_poses_incremental, force_loop_closure
from src.sfm.triangulation import triangulate_all_points, merge_triangulated_points
from src.sfm.bundle_adjustment import run_global_ba
from src.dense_reconstruction.mvs import process_mvs
from src.dense_reconstruction.point_cloud import process_dense_reconstruction, create_surface_mesh, save_mesh
from src.surface_reconstruction.mesh_generation import process_point_cloud_to_mesh, clean_mesh
from src.surface_reconstruction.texture_mapping import create_textured_mesh_from_point_cloud


from src.visualization.plot_matches import plot_matches, plot_feature_matching_analysis
from src.visualization.point_cloud_visualizer import plot_interactive_point_cloud, create_point_cloud_animation
from src.visualization.camera_visualizer import plot_interactive_camera_poses
from src.visualization.mesh_visualizer import visualize_mesh_o3d, plot_interactive_mesh

# Set up matplotlib for inline display
%matplotlib inline
# For newer matplotlib versions (3.6+)
plt.style.use('default')

In [ ]:
# Define configuration
config = {
    "preprocessing": {
        "resize_max_dimension": 1000,
        "enhance_contrast": True
    },
    "features": {
        "method": "sift",
        "max_features": 50000,
        "use_multiscale": True,
        "use_dense": True,
        "dense_step": 5,
        "contrast_threshold": 0.01,
        "edge_threshold": 8
    },
    "matching": {
        "ratio_threshold": 0.7,
        "geometric_verification": True,
        "min_matches": 10,
        "verification_method": "fundamental",
        "ransac_threshold": 2.0,
        "cross_check": True,
        "max_epipolar_error": 1.0,
        "confidence": 0.999
    },
    "calibration": {
        "focal_length_factor": 1.3,
        "principal_point": "center",
        "refine_intrinsics": True
    },
    "sfm": {
        "incremental": True,
        "refine_poses": True,
        "min_triangulation_angle_deg": 2.0,
        "reprojection_error_threshold": 2.0,
        "bundle_adjustment_max_iterations": 100,
        "max_reprojection_error": 3.0,
        "merge_threshold": 0.001
    },
    "mvs": {
        "min_disparity": 0,
        "num_disparities": 128,
        "block_size": 7,
        "filter_depths": True,
        "consistency_threshold": 0.01,
        "num_source_views": 5
    },
    "point_cloud": {
        "voxel_size": 0.02,
        "nb_neighbors": 30,
        "std_ratio": 1.5,
        "confidence_threshold": 0.8
    },
    "surface": {
        "method": "poisson",
        "depth": 10,
        "cleanup": True,
        "trim": 7.0
    },
    "visualization": {
        "point_size": 2,
        "camera_size": 6,
        "point_color_method": "rgb"
    }
}

# Create output directory for results
output_dir = "../data/results/"
os.makedirs(output_dir, exist_ok=True)

print("Configuration loaded with 50,000 maximum features and strict quality filtering.")

In [ ]:
# Define paths to both datasets
dataset_path_black = '../data/dinosaur_cropped_black/'  # Black background
dataset_path_original = '../data/dinosaur_cropped/'       # Original background

# Load black background images for feature extraction and matching
print("Loading black background images for feature extraction...")
black_images = load_image_sequence(dataset_path_black, pattern="viff.*.png")
print(f"Loaded {len(black_images)} black background images.")

# Also load original background images for depth estimation
print("Loading original background images for depth estimation...")
original_images = load_image_sequence(dataset_path_original, pattern="viff.*.png")
print(f"Loaded {len(original_images)} original background images.")

# Make sure image lists are the same length and in the same order
if len(black_images) != len(original_images):
    print("Warning: Different number of images in the two datasets")
    # Keep only matching filenames
    black_filenames = [filename for _, filename in black_images]
    original_filenames = [filename for _, filename in original_images]
    common_filenames = set(black_filenames).intersection(set(original_filenames))
    
    black_images = [(img, filename) for img, filename in black_images if filename in common_filenames]
    original_images = [(img, filename) for img, filename in original_images if filename in common_filenames]
    
    print(f"Using {len(black_images)} images that exist in both datasets")

# Resize black background images
max_dim = config['preprocessing']['resize_max_dimension']
black_images = resize_images(black_images, max_dimension=max_dim)
original_images = resize_images(original_images, max_dimension=max_dim)
print(f"Resized images to maximum dimension of {max_dim} pixels.")

# Enhance contrast if specified
if config['preprocessing']['enhance_contrast']:
    black_images = enhance_contrast(black_images)
    original_images = enhance_contrast(original_images)
    print("Enhanced image contrast.")

# Display a comparison between black and original images
n_images = min(3, len(black_images))
fig, axes = plt.subplots(2, n_images, figsize=(15, 8))

for i in range(n_images):
    img_black, filename_black = black_images[i]
    axes[0, i].imshow(img_black)
    axes[0, i].set_title(f"Black BG: {filename_black}")
    axes[0, i].axis('off')
    
    img_orig, filename_orig = original_images[i]
    axes[1, i].imshow(img_orig)
    axes[1, i].set_title(f"Original BG: {filename_orig}")
    axes[1, i].axis('off')

plt.tight_layout()
plt.show()

In [4]:
def filter_matches_on_black_background(matches_dict, images):
    """
    Filter out matches where either keypoint is on a black background.
    
    Args:
        matches_dict: Dictionary of matches from match_image_pairs
        images: List of (image, filename) tuples
        
    Returns:
        Filtered matches dictionary
    """
    # Create a mapping from filename to image for quick lookup
    image_dict = {filename: img for img, filename in images}
    
    # Create masks for each image (non-black regions)
    mask_dict = {}
    for filename, img in image_dict.items():
        if len(img.shape) == 3:
            gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        else:
            gray = img
        # Create binary mask - pixels with value > 15 are considered part of the dinosaur
        _, mask = cv2.threshold(gray, 15, 255, cv2.THRESH_BINARY)
        mask_dict[filename] = mask
    
    filtered_matches_dict = {}
    
    # For each image pair in the matches dictionary
    for (img1_name, img2_name), (kp1, kp2, matches) in matches_dict.items():
        mask1 = mask_dict[img1_name]
        mask2 = mask_dict[img2_name]
        
        # Filter matches - keep only those where both points are on non-black areas
        filtered_matches = []
        for m in matches:
            # Get keypoint coordinates (need to round to integers for mask indexing)
            x1, y1 = int(round(kp1[m.queryIdx].pt[0])), int(round(kp1[m.queryIdx].pt[1]))
            x2, y2 = int(round(kp2[m.trainIdx].pt[0])), int(round(kp2[m.trainIdx].pt[1]))
            
            # Check if coordinates are within image bounds
            h1, w1 = mask1.shape
            h2, w2 = mask2.shape
            
            if (0 <= x1 < w1 and 0 <= y1 < h1 and 0 <= x2 < w2 and 0 <= y2 < h2):
                # Check if both points are on the dinosaur (non-black regions)
                if mask1[y1, x1] > 0 and mask2[y2, x2] > 0:
                    filtered_matches.append(m)
        
        # If we have enough matches after filtering, keep this pair
        if len(filtered_matches) >= 8:  # Use 8 as minimum for fundamental matrix estimation
            filtered_matches_dict[(img1_name, img2_name)] = (kp1, kp2, filtered_matches)
            print(f"Filtered {img1_name}-{img2_name}: {len(matches)} → {len(filtered_matches)} matches")
    
    print(f"Filtered matches: {len(matches_dict)} pairs → {len(filtered_matches_dict)} pairs")
    return filtered_matches_dict

In [ ]:
# Extract features from black background images with enhanced strategies
print("\nExtracting comprehensive features from black background images...")
feature_method = config['features']['method']
max_features = config['features']['max_features']  # Triple the feature count

# Set SIFT extraction parameters if using SIFT
if feature_method.lower() == 'sift':
    print(f"Using enhanced SIFT extraction with target of {max_features} features per image")
    
    # Extract features using enhanced extraction methods
    features_dict = extract_features_from_image_set(black_images, method=feature_method, n_features=max_features, contrast_threshold=config['features']['contrast_threshold'])
else:
    # For other methods, use standard extraction
    features_dict = extract_features_from_image_set(black_images, method=feature_method, n_features=max_features, contrast_threshold=config['features']['contrast_threshold'])

# Print feature counts
total_features = 0
for filename, (keypoints, descriptors) in features_dict.items():
    print(f"{filename}: {len(keypoints)} keypoints")
    total_features += len(keypoints)

print(f"Total features extracted: {total_features} (avg {total_features/len(features_dict):.0f} per image)")

# Visualize keypoints on a sample image
sample_img, sample_filename = black_images[0]
sample_keypoints, _ = features_dict[sample_filename]

# Plot keypoints
fig, ax = plt.subplots(figsize=(10, 8))
ax.imshow(cv2.drawKeypoints(sample_img, sample_keypoints, None, 
                          color=(0, 255, 0), flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS))
ax.set_title(f"{feature_method.upper()} Features: {sample_filename} ({len(sample_keypoints)} keypoints)")
ax.axis('off')
plt.tight_layout()
plt.show()

# Create image pairs for matching with a comprehensive circular strategy
print("\nCreating optimized view graph for circular object reconstruction...")
filenames = sorted([filename for _, filename in black_images], 
                  key=lambda x: int(''.join(filter(str.isdigit, x))))

# Number of images
n = len(filenames)
print(f"Creating view graph for {n} images")

# Create image pairs for matching with a comprehensive circular strategy
image_pairs = []

# 1. Keep sequential pairs as your foundation
for i in range(n-1):
    image_pairs.append((filenames[i], filenames[i+1]))
image_pairs.append((filenames[-1], filenames[0]))  # Close the loop
# Remove duplicates while preserving order
seen = set()
image_pairs = [x for x in image_pairs if not (x in seen or seen.add(x))]

print(f"Created {len(image_pairs)} image pairs for matching")

# Match features
matches_dict = match_image_pairs(
    features_dict, 
    image_pairs, 
    ratio_threshold=config['matching']['ratio_threshold'],
    geometric_verify=config['matching']['geometric_verification'],
    min_matches=config['matching']['min_matches']
)

matches_dict = filter_matches_on_black_background(matches_dict, black_images)


print(f"Successfully matched {len(matches_dict)} image pairs.")

# Display matches for a sample pair
if len(matches_dict) > 0:
    # Choose a sample pair
    sample_pair = list(matches_dict.keys())[0]
    img1_name, img2_name = sample_pair
    kp1, kp2, matches = matches_dict[sample_pair]
    
    # Get the images
    img1 = next(img for img, filename in black_images if filename == img1_name)
    img2 = next(img for img, filename in black_images if filename == img2_name)
    
    # Plot matches
    plot_matches(img1, kp1, img2, kp2, matches[:100],  # Only plot first 100 matches for clarity
                title=f"Matches between {img1_name} and {img2_name}: {len(matches)} matches")

    # Show matching statistics across all pairs
    plot_feature_matching_analysis(matches_dict, figsize=(12, 6))

In [12]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

# Define parameter combinations
ratio_thresholds = [0.75]
min_matches_values = [10, 12, 14, 16, 18, 20]

# Dictionary to store camera poses for each parameter combination
all_camera_poses = {}

# Estimate camera intrinsics (same for all runs)
sample_img, _ = black_images[0]
image_shape = sample_img.shape
focal_length_factor = config['calibration']['focal_length_factor']
focal_length = focal_length_factor * max(image_shape[0], image_shape[1])
K = estimate_camera_matrix(image_shape, focal_length)
print("Estimated camera matrix:")
print(K)

# Run for each parameter combination and show interactive plot immediately
for ratio_threshold in ratio_thresholds:
    for min_matches in min_matches_values:
        print(f"\n{'='*60}")
        print(f"Processing ratio_threshold={ratio_threshold}, min_matches={min_matches}")
        print(f"{'='*60}")
        
        # Run matching
        matches_dict = match_image_pairs(
            features_dict, 
            image_pairs, 
            ratio_threshold=ratio_threshold,
            geometric_verify=config['matching']['geometric_verification'],
            min_matches=min_matches
        )
        
        # Filter matches
        matches_dict = filter_matches_on_black_background(matches_dict, black_images)
        
        # Estimate camera poses
        camera_poses = estimate_poses_incremental(matches_dict, K, min_matches)
        
        print(f"Estimated poses for {len(camera_poses)} cameras.")
        
        # Store camera poses for this parameter combination
        all_camera_poses[(ratio_threshold, min_matches)] = camera_poses
        
        # Show the interactive plot immediately for this parameter combination
        if len(camera_poses) > 0:
            print("Displaying interactive 3D plot...")
            plot_interactive_camera_poses(camera_poses)
        else:
            print("No camera poses were estimated with these parameters. Skipping plot.")

# Print summary table at the end
print("\nSummary of results:")
print("-" * 60)
print(f"{'Ratio':<10} {'Min Matches':<15} {'Num Cameras':<15}")
print("-" * 60)
for (ratio, min_m), poses in all_camera_poses.items():
    print(f"{ratio:<10} {min_m:<15} {len(poses):<15}")

Estimated camera matrix:
[[903.5   0.  347.5]
 [  0.  903.5 286.5]
 [  0.    0.    1. ]]

Processing ratio_threshold=0.75, min_matches=10
Filtered viff.000.png-viff.001.png: 2176 → 2112 matches
Filtered viff.001.png-viff.002.png: 2052 → 2002 matches
Filtered viff.002.png-viff.003.png: 2140 → 2104 matches
Filtered viff.003.png-viff.004.png: 2354 → 2325 matches
Filtered viff.004.png-viff.005.png: 1900 → 1872 matches
Filtered viff.005.png-viff.006.png: 1816 → 1797 matches
Filtered viff.006.png-viff.007.png: 1657 → 1634 matches
Filtered viff.007.png-viff.008.png: 1565 → 1554 matches
Filtered viff.008.png-viff.009.png: 1615 → 1591 matches
Filtered viff.009.png-viff.010.png: 1578 → 1564 matches
Filtered viff.010.png-viff.011.png: 1756 → 1737 matches
Filtered viff.011.png-viff.012.png: 1676 → 1654 matches
Filtered viff.012.png-viff.013.png: 1626 → 1608 matches
Filtered viff.013.png-viff.014.png: 1418 → 1408 matches
Filtered viff.014.png-viff.015.png: 1341 → 1329 matches
Filtered viff.015.png-


Processing ratio_threshold=0.75, min_matches=12
Filtered viff.000.png-viff.001.png: 2177 → 2112 matches
Filtered viff.001.png-viff.002.png: 2231 → 2181 matches
Filtered viff.002.png-viff.003.png: 2361 → 2317 matches
Filtered viff.003.png-viff.004.png: 2279 → 2246 matches
Filtered viff.004.png-viff.005.png: 2187 → 2155 matches
Filtered viff.005.png-viff.006.png: 1822 → 1802 matches
Filtered viff.006.png-viff.007.png: 1624 → 1605 matches
Filtered viff.007.png-viff.008.png: 1531 → 1518 matches
Filtered viff.008.png-viff.009.png: 1605 → 1583 matches
Filtered viff.009.png-viff.010.png: 1418 → 1411 matches
Filtered viff.010.png-viff.011.png: 1683 → 1664 matches
Filtered viff.011.png-viff.012.png: 1674 → 1652 matches
Filtered viff.012.png-viff.013.png: 1646 → 1628 matches
Filtered viff.013.png-viff.014.png: 1482 → 1471 matches
Filtered viff.014.png-viff.015.png: 1366 → 1353 matches
Filtered viff.015.png-viff.016.png: 1267 → 1251 matches
Filtered viff.016.png-viff.017.png: 1444 → 1425 matches


Processing ratio_threshold=0.75, min_matches=14
Filtered viff.000.png-viff.001.png: 2208 → 2144 matches
Filtered viff.001.png-viff.002.png: 2064 → 2025 matches
Filtered viff.002.png-viff.003.png: 2333 → 2286 matches
Filtered viff.003.png-viff.004.png: 2348 → 2312 matches
Filtered viff.004.png-viff.005.png: 2168 → 2135 matches
Filtered viff.005.png-viff.006.png: 1868 → 1849 matches
Filtered viff.006.png-viff.007.png: 1649 → 1627 matches
Filtered viff.007.png-viff.008.png: 1604 → 1592 matches
Filtered viff.008.png-viff.009.png: 1558 → 1538 matches
Filtered viff.009.png-viff.010.png: 1585 → 1573 matches
Filtered viff.010.png-viff.011.png: 1704 → 1685 matches
Filtered viff.011.png-viff.012.png: 1683 → 1661 matches
Filtered viff.012.png-viff.013.png: 1628 → 1611 matches
Filtered viff.013.png-viff.014.png: 1529 → 1516 matches
Filtered viff.014.png-viff.015.png: 1216 → 1205 matches
Filtered viff.015.png-viff.016.png: 1267 → 1246 matches
Filtered viff.016.png-viff.017.png: 1491 → 1472 matches


Processing ratio_threshold=0.75, min_matches=16
Filtered viff.000.png-viff.001.png: 2159 → 2095 matches
Filtered viff.001.png-viff.002.png: 2221 → 2172 matches
Filtered viff.002.png-viff.003.png: 2191 → 2148 matches
Filtered viff.003.png-viff.004.png: 2211 → 2180 matches
Filtered viff.004.png-viff.005.png: 2174 → 2141 matches
Filtered viff.005.png-viff.006.png: 1877 → 1859 matches
Filtered viff.006.png-viff.007.png: 1752 → 1729 matches
Filtered viff.007.png-viff.008.png: 1578 → 1566 matches
Filtered viff.008.png-viff.009.png: 1560 → 1539 matches
Filtered viff.009.png-viff.010.png: 1542 → 1527 matches
Filtered viff.010.png-viff.011.png: 1623 → 1603 matches
Filtered viff.011.png-viff.012.png: 1610 → 1591 matches
Filtered viff.012.png-viff.013.png: 1656 → 1639 matches
Filtered viff.013.png-viff.014.png: 1407 → 1391 matches
Filtered viff.014.png-viff.015.png: 1386 → 1374 matches
Filtered viff.015.png-viff.016.png: 1339 → 1317 matches
Filtered viff.016.png-viff.017.png: 1464 → 1444 matches


Processing ratio_threshold=0.75, min_matches=18
Filtered viff.000.png-viff.001.png: 2087 → 2038 matches
Filtered viff.001.png-viff.002.png: 2211 → 2166 matches
Filtered viff.002.png-viff.003.png: 2164 → 2126 matches
Filtered viff.003.png-viff.004.png: 2160 → 2128 matches
Filtered viff.004.png-viff.005.png: 2088 → 2056 matches
Filtered viff.005.png-viff.006.png: 1708 → 1691 matches
Filtered viff.006.png-viff.007.png: 1713 → 1690 matches
Filtered viff.007.png-viff.008.png: 1614 → 1601 matches
Filtered viff.008.png-viff.009.png: 1585 → 1562 matches
Filtered viff.009.png-viff.010.png: 1549 → 1542 matches
Filtered viff.010.png-viff.011.png: 1743 → 1724 matches
Filtered viff.011.png-viff.012.png: 1604 → 1584 matches
Filtered viff.012.png-viff.013.png: 1656 → 1640 matches
Filtered viff.013.png-viff.014.png: 1501 → 1488 matches
Filtered viff.014.png-viff.015.png: 1377 → 1364 matches
Filtered viff.015.png-viff.016.png: 1292 → 1271 matches
Filtered viff.016.png-viff.017.png: 1425 → 1412 matches


Processing ratio_threshold=0.75, min_matches=20
Filtered viff.000.png-viff.001.png: 2162 → 2106 matches
Filtered viff.001.png-viff.002.png: 2208 → 2160 matches
Filtered viff.002.png-viff.003.png: 2296 → 2251 matches
Filtered viff.003.png-viff.004.png: 2196 → 2165 matches
Filtered viff.004.png-viff.005.png: 2154 → 2125 matches
Filtered viff.005.png-viff.006.png: 1773 → 1756 matches
Filtered viff.006.png-viff.007.png: 1748 → 1725 matches
Filtered viff.007.png-viff.008.png: 1513 → 1503 matches
Filtered viff.008.png-viff.009.png: 1690 → 1667 matches
Filtered viff.009.png-viff.010.png: 1557 → 1544 matches
Filtered viff.010.png-viff.011.png: 1755 → 1736 matches
Filtered viff.011.png-viff.012.png: 1629 → 1609 matches
Filtered viff.012.png-viff.013.png: 1643 → 1625 matches
Filtered viff.013.png-viff.014.png: 1458 → 1448 matches
Filtered viff.014.png-viff.015.png: 1328 → 1317 matches
Filtered viff.015.png-viff.016.png: 1236 → 1220 matches
Filtered viff.016.png-viff.017.png: 1462 → 1445 matches


Summary of results:
------------------------------------------------------------
Ratio      Min Matches     Num Cameras    
------------------------------------------------------------
0.75       10              36             
0.75       12              36             
0.75       14              36             
0.75       16              36             
0.75       18              36             
0.75       20              36             


In [9]:
# Triangulate 3D points
print("\nTriangulating 3D points...")
points_3d, point_observations = triangulate_all_points(camera_poses, matches_dict, K)

print(f"Triangulated {len(points_3d)} 3D points.")

# Merge close points to reduce noise
merged_points, merged_observations = merge_triangulated_points(points_3d, point_observations, threshold=0.002)
print(f"After merging: {len(merged_points)} 3D points.")

# Bundle adjustment to refine camera poses and 3D points
if config['sfm']['refine_poses'] and len(merged_points) > 0:
    print("\nRunning bundle adjustment...")
    refined_poses, refined_points, _ = run_global_ba(camera_poses, matches_dict, K, iterations=50)
    points_3d = refined_points
    print("Bundle adjustment complete.")

plot_interactive_camera_poses(refined_poses)

# Visualize sparse point cloud
if len(points_3d) > 0:
    print("\nVisualizing sparse point cloud...")
    points_array = np.array(points_3d)
    
    # Assign random colors for visualization
    np.random.seed(42)  # For reproducibility
    colors = np.random.rand(len(points_array), 3)
    
    # Interactive visualization
    plot_interactive_point_cloud(points_array, colors, title="Sparse 3D Reconstruction")

    # Save sparse point cloud
    sparse_cloud_file = os.path.join(output_dir, "dinosaur_sparse.ply")
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points_array)
    pcd.colors = o3d.utility.Vector3dVector(colors)
    o3d.io.write_point_cloud(sparse_cloud_file, pcd)
    print(f"Saved sparse point cloud to {sparse_cloud_file}")


Triangulating 3D points...
Triangulating from 36 camera poses and 36 feature matches
Found 36 valid image pairs for triangulation
Triangulated 3523 3D points
Triangulated 3523 3D points.
Merged 3523 points into 2643 points
After merging: 2643 3D points.

Running bundle adjustment...
Triangulating from 36 camera poses and 36 feature matches
Found 36 valid image pairs for triangulation
Triangulated 3523 3D points
Merged 3523 points into 2644 points
Prepared 2644 points with at least 2 observations for bundle adjustment
   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         3.0360e+03                                    7.47e+05    
       1              3         1.2886e+03      1.75e+03       4.09e+01       8.03e+04    
       2              4         8.3057e+02      4.58e+02       8.27e+01       3.91e+04    
       3              5         4.4735e+02      3.83e+02       5.10e+01       1.22e+04    
       4             


Visualizing sparse point cloud...


Saved sparse point cloud to ../data/results/dinosaur_sparse.ply


In [ ]:
# Significantly modified MVS configuration optimized for challenging objects like dinosaur figurines
improved_mvs_config = {
    'min_disparity': 0,
    'num_disparities': 192,      # Increased for better depth range coverage
    'block_size': 13,            # Larger block for smooth surfaces
    'filter_depths': True,
    'consistency_threshold': 0.18, # Much more permissive threshold for challenging textures
    'num_source_views': 6,       # Use more source views
    'speckle_size': 150,         # Increased speckle filtering
    'speckle_range': 3,          # Slightly increased range
    'uniqueness_ratio': 5,       # Lower ratio to capture more potential matches
    'pre_filter_cap': 31
}

# Run MVS with improved parameters using hybrid approach - both clean and original images
print("\nRunning Multi-View Stereo with hybrid approach...")

# Create hybrid image list that combines black background and original images
hybrid_images = []
for name in camera_poses.keys():
    # First find the original background image
    orig_img = next((img for img, filename in original_images if filename == name), None)
    # Also find the black background image
    black_img = next((img for img, filename in black_images if filename == name), None)
    
    if orig_img is not None and black_img is not None:
        # Create a mask from the black background image
        if len(black_img.shape) == 3:
            gray = cv2.cvtColor(black_img, cv2.COLOR_RGB2GRAY)
        else:
            gray = black_img
        
        # Create binary mask (non-black areas)
        _, mask = cv2.threshold(gray, 10, 255, cv2.THRESH_BINARY)
        
        # Add preprocessing to enhance texture
        # Apply CLAHE to original image to enhance texture
        if len(orig_img.shape) == 3:
            lab = cv2.cvtColor(orig_img, cv2.COLOR_RGB2LAB)
            l, a, b = cv2.split(lab)
            clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
            cl = clahe.apply(l)
            enhanced_lab = cv2.merge((cl, a, b))
            enhanced_orig = cv2.cvtColor(enhanced_lab, cv2.COLOR_LAB2RGB)
        else:
            clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
            enhanced_orig = clahe.apply(orig_img)
        
        # Use the original image (with texture enhancement) for MVS
        hybrid_images.append((enhanced_orig, name))
    elif orig_img is not None:
        # Fall back to original if no black background version exists
        hybrid_images.append((orig_img, name))
    elif black_img is not None:
        # Fall back to black background if no original version exists
        hybrid_images.append((black_img, name))

mvs_results = process_mvs(hybrid_images, camera_poses, K, config['mvs'])

# Extract depth maps
depth_maps = mvs_results['filtered_depth_maps']
confidence_maps = mvs_results['confidence_maps']

print(f"Generated {len(depth_maps)} depth maps.")

# Apply additional post-processing to improve depth maps
print("Applying additional depth map enhancement...")
enhanced_depth_maps = {}

for name, depth_map in depth_maps.items():
    # Find valid depths
    valid_mask = depth_map > 0
    
    if np.sum(valid_mask) > 0:
        # Apply median filter to remove noise (with larger kernel)
        filtered_depth = cv2.medianBlur(depth_map, 5)
        
        # Fill small holes using morphological operations
        kernel = np.ones((5, 5), np.uint8)
        valid_mask_filled = cv2.morphologyEx(valid_mask.astype(np.uint8), cv2.MORPH_CLOSE, kernel)
        
        # Use a more aggressive bilateral filter to smooth while preserving edges
        smoothed_depth = cv2.bilateralFilter(filtered_depth, 9, 75, 75)
        
        # Combine the results
        enhanced_depth = smoothed_depth.copy()
        
        # Keep only the valid regions after morphological operations
        enhanced_depth[valid_mask_filled == 0] = 0
        
        enhanced_depth_maps[name] = enhanced_depth
    else:
        enhanced_depth_maps[name] = depth_map

# Replace the original depth maps with enhanced ones
depth_maps = enhanced_depth_maps

# Visualize a sample depth map
if len(depth_maps) > 0:
    # Choose a sample depth map
    sample_name = list(depth_maps.keys())[0]
    depth_map = depth_maps[sample_name]
    
    # Normalize depth for visualization
    valid_mask = depth_map > 0
    if np.any(valid_mask):
        min_depth = np.min(depth_map[valid_mask])
        max_depth = np.max(depth_map[valid_mask])
        normalized_depth = np.zeros_like(depth_map)
        normalized_depth[valid_mask] = (depth_map[valid_mask] - min_depth) / (max_depth - min_depth)
        
        # Display depth map with improved color mapping
        plt.figure(figsize=(10, 8))
        plt.imshow(normalized_depth, cmap='turbo')  # Using turbo colormap for better depth visualization
        plt.colorbar(label='Normalized Depth')
        plt.title(f"Enhanced Depth Map: {sample_name}")
        plt.axis('off')
        plt.show()
        
        # Also show the confidence map
        conf_map = confidence_maps[sample_name]
        plt.figure(figsize=(10, 8))
        plt.imshow(conf_map, cmap='viridis')
        plt.colorbar(label='Confidence')
        plt.title(f"Confidence Map: {sample_name}")
        plt.axis('off')
        plt.show()

# Enhanced point cloud settings specifically for challenging objects like dinosaur models
enhanced_point_cloud_config = {
    'voxel_size': 0.02,           # Smaller voxel size for more detail
    'nb_neighbors': 30,           # More neighbors for better outlier detection
    'std_ratio': 2.5,             # More permissive outlier rejection
    'confidence_threshold': 0.3    # Lower threshold to include more points
}

# Generate dense point cloud with size limits
print("\nGenerating dense point cloud from depth maps...")
dense_results = process_dense_reconstruction(
    hybrid_images, camera_poses, K, depth_maps, confidence_maps, enhanced_point_cloud_config)

# Extract results
dense_points = dense_results['filtered_points']
dense_colors = dense_results['filtered_colors']

# Limit point cloud size if needed
if len(dense_points) > 500000:
    print(f"Point cloud too large ({len(dense_points)} points), downsampling to ~500K points...")
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(dense_points)
    pcd.colors = o3d.utility.Vector3dVector(dense_colors)
    
    # Gradually increase voxel size until points are under 500K
    target_size = 500000
    voxel_size = 0.025
    while len(pcd.points) > target_size and voxel_size < 0.1:
        voxel_size += 0.01
        pcd = pcd.voxel_down_sample(voxel_size)
        print(f"Downsampled to {len(pcd.points)} points with voxel size {voxel_size:.2f}")
    
    dense_points = np.asarray(pcd.points)
    dense_colors = np.asarray(pcd.colors)

if len(dense_points) > 0:
    print(f"Final point cloud contains {len(dense_points)} points.")
    
    # Visualize dense point cloud
    plot_interactive_point_cloud(dense_points, dense_colors, title="Dense 3D Reconstruction")
    
    # Save dense point cloud
    dense_cloud_file = os.path.join(output_dir, "dinosaur_dense.ply")
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(dense_points)
    pcd.colors = o3d.utility.Vector3dVector(dense_colors)
    o3d.io.write_point_cloud(dense_cloud_file, pcd)
    print(f"Saved dense point cloud to {dense_cloud_file}")
    
    # Create mesh with optimized parameters for dinosaur model
    mesh_config = {
        'method': 'poisson',
        'depth': 9,             # Higher depth for more detail
        'scale': 1.1,           # Slightly aggressive weighing
        'cleanup': True
    }
    
    print("\nCreating mesh from point cloud...")
    # Compute normals with better parameters
    print("Computing normals with optimized parameters...")
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(dense_points)
    pcd.colors = o3d.utility.Vector3dVector(dense_colors)
    
    # Use more neighbors for smoother normals on the dinosaur surface
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
    pcd.orient_normals_consistent_tangent_plane(k=20)
    
    normals = np.asarray(pcd.normals)
    
    # Create mesh
    mesh = create_surface_mesh(
        dense_points, 
        dense_colors, 
        normals,
        method=mesh_config['method'], 
        depth=mesh_config['depth']
    )
    
    if mesh:
        # Clean the mesh
        print("Cleaning mesh to improve quality...")
        mesh = clean_mesh(mesh, detail_level=3)
        
        # Optional: apply Laplacian smoothing to get smoother dinosaur surface
        print("Applying final smoothing to improve surface quality...")
        mesh = mesh.filter_smooth_taubin(number_of_iterations=10)
        
        # Save the mesh
        mesh_file = os.path.join(output_dir, "dinosaur_mesh.ply")
        save_mesh(mesh, mesh_file)
        print(f"Saved mesh to {mesh_file}")
        
        # Visualize the mesh
        visualize_mesh_o3d(mesh, window_name="Dinosaur 3D Mesh")

In [ ]:
# Generate mesh from dense point cloud
print("\nGenerating surface mesh...")
mesh, pcd = process_point_cloud_to_mesh(
    dense_points, dense_colors, method=config['surface']['method'])

# Display mesh statistics
print(f"Mesh contains {len(mesh.vertices)} vertices and {len(mesh.triangles)} triangles.")

# Visualize mesh (this will open an Open3D window)
print("\nVisualizing mesh (this will open a new window)...")
visualize_mesh_o3d(mesh, window_name="Reconstructed Mesh")

# Save mesh
mesh_file = os.path.join(output_dir, "dinosaur_mesh.ply")
o3d.io.write_triangle_mesh(mesh_file, mesh)
print(f"Saved mesh to {mesh_file}")

In [ ]:
# Generate textured mesh
print("\nGenerating textured mesh...")
textured_mesh = create_textured_mesh_from_point_cloud(
    dense_points, dense_colors, images, camera_poses, K, 
    reconstruction_method=config['surface']['method'])

# Visualize textured mesh (this will open an Open3D window)
print("\nVisualizing textured mesh (this will open a new window)...")
visualize_mesh_o3d(textured_mesh, window_name="Textured Mesh")

# Save textured mesh
textured_mesh_file = os.path.join(output_dir, "dinosaur_textured.obj")
o3d.io.write_triangle_mesh(textured_mesh_file, textured_mesh)
print(f"Saved textured mesh to {textured_mesh_file}")

In [ ]:
# Create interactive mesh visualization using Plotly
print("\nCreating interactive mesh visualization...")

# Extract mesh data for Plotly
vertices = np.asarray(mesh.vertices)
triangles = np.asarray(mesh.triangles)
vertex_colors = np.asarray(mesh.vertex_colors) if mesh.has_vertex_colors() else None

# Create interactive plot
plot_interactive_mesh(vertices, triangles, vertex_colors, title="Interactive 3D Dinosaur Model")

# Create rotating animation (optional)
print("\nCreating 360° animation of the model...")
animation = create_point_cloud_animation(
    dense_points, dense_colors, n_frames=36, 
    output_file=os.path.join(output_dir, "dinosaur_animation.html"))

print("\n3D reconstruction pipeline complete!")
print(f"All results saved to: {output_dir}")

# Display summary of the reconstruction
print("\nReconstruction Summary:")
print(f"Images processed: {len(images)}")
print(f"Camera poses estimated: {len(camera_poses)}")
print(f"Sparse points: {len(points_3d)}")
print(f"Dense points: {len(dense_points)}")
print(f"Mesh vertices: {len(mesh.vertices)}")
print(f"Mesh triangles: {len(mesh.triangles)}")

In [ ]:
# Simplify and optimize mesh for 3D printing (optional)
print("\nOptimizing mesh for export...")

# Make a copy of the mesh for optimization
export_mesh = o3d.geometry.TriangleMesh(mesh)

# Remove any non-manifold edges
export_mesh.remove_non_manifold_edges()

# Fill holes
export_mesh = export_mesh.filter_smooth_simple(5)  # Smooth mesh

# Simplify mesh to reduce polygon count
target_triangles = int(len(export_mesh.triangles) * 0.5)  # Reduce to 50%
export_mesh = export_mesh.simplify_quadric_decimation(target_triangles)

# Fix mesh normals
export_mesh.compute_vertex_normals()

# Save optimized mesh in multiple formats
o3d.io.write_triangle_mesh(os.path.join(output_dir, "dinosaur_optimized.obj"), export_mesh)
o3d.io.write_triangle_mesh(os.path.join(output_dir, "dinosaur_optimized.stl"), export_mesh)

print("Optimized mesh exported in OBJ and STL formats, suitable for 3D printing and other applications.")